In [3]:
import ezdxf
from dxfgrouping import filterlayerelements, makemergevertset, linearizeelement
from dxfgrouping import getblockcomponent, dxfoutputblocks

def convertall(inputfilename, outputfilename, cutlayers, penlayers, breflY, dmax):
    d = ezdxf.readfile(inputfilename)
    print("layerspresent:", set(e.dxf.layer for e in d.entities))
    d = ezdxf.readfile(inputfilename)
    lwpolylines = [ e  for e in d.entities  if e.dxftype() == "LWPOLYLINE" ]
    if lwpolylines:
        #print("exploding %d lwpolylines" % len(lwpolylines))
        for e in lwpolylines:
            e.explode()

    cutelements = filterlayerelements(d, cutlayers)
    penelements = filterlayerelements(d, penlayers)
        
    lcutelements, lpenelements = cutelements, penelements
    elementgroups = [ ]

    while lcutelements:
        res = getblockcomponent(lcutelements, lpenelements, dmax)
        (outercutelements, outercutelementsdir) = res[0]
        (internalcutelements, internalpenelements) = res[1]
        (remainingcutelements, remainingpenelements) = res[2]
        elementgroups.append((outercutelements, outercutelementsdir, internalcutelements, internalpenelements))
        lcutelements, lpenelements = remainingcutelements, remainingpenelements
        #print("Component: ", len(outercutelements), len(internalcutelements), len(internalpenelements))
        
    dxfoutputblocks(outputfilename, elementgroups, breflY)


In [4]:
import os
def createorclear(aamaoutdir, lsubdir):
    subdir = os.path.join(aamaoutdir, lsubdir)
    if os.path.exists(subdir):
        for f in os.listdir(subdir):
            os.remove(os.path.join(subdir, f))
    else:
        os.mkdir(subdir)
    return subdir
        


In [5]:
#-F15-BS-01- Bottom surface. 
#  BS-CUT layer to be cut, BS-PLOT to be plotted, all pieces cut from '170 color'

# All need left and right hand versions.

aamaoutdir = "../dxffiles/aamaout"
dmax = 0.2

inputfilename = "../dxffiles/F15-BS-01.dxf"
d170color = createorclear(aamaoutdir, "170color")
cutlayers, penlayers = ["BS-CUT"], ['BS-PLOT']
outputfilename = os.path.join(d170color, "F15-BS.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
outputfilename = os.path.join(d170color, "F15-BS-reflY.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)


layerspresent: {'BS-PLOT', 'CONSTRUCTION', 'BS-CUT', 'REFERENCE', 'HISTORY'}
dmaxs-tail:  [6.245036057586848e-12, 1.0999348430804918e-11, 0.06873168711912457]
layerspresent: {'BS-PLOT', 'CONSTRUCTION', 'BS-CUT', 'REFERENCE', 'HISTORY'}
dmaxs-tail:  [6.245036057586848e-12, 1.0999348430804918e-11, 0.06873168711912457]


In [8]:
#-F15-TSF-Master-04.dxf. Top surface front (leading edge). 
#  TSF-CUT and TSF-PLOT to be cut from '210 mast sleeve' 
#  PATCH-NORM CUT and PLOT to be from 210 mast sleeve if making a standard glider
#    or PATCH-CARR_CUT and PLOT to be from 210 mast sleeve if making a carrier wing one. 
#  MYLAR-CUT to cut from Mylar.

aamaoutdir = "../dxffiles/aamaout"
dmax = 0.2

carrierwingglider = False

inputfilename = "../dxffiles/F15-TSF-Master-04.dxf"
d210mastsleeve = createorclear(aamaoutdir, "210mastsleeve")
cutlayers, penlayers = ['TSF-CUT'], ['TSF-PLOT']
outputfilename = os.path.join(d210mastsleeve, "F15-TSF.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
outputfilename = os.path.join(d210mastsleeve, "F15-TSF-reflY.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)

if carrierwingglider:
    cutlayers, penlayers = ['PATCH-CARR-CUT'], ['PATCH-CARR-PLOT']
else:
    cutlayers, penlayers = ['PATCH-NORM-CUT'], ['PATCH-NORM-PLOT']
outputfilename = os.path.join(d210mastsleeve, "F15-TSF-PATCH.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
outputfilename = os.path.join(d210mastsleeve, "F15-TSF-PATCH-reflY.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)

cutlayers, penlayers = ['MYLAR-CUT'], []
dmylar = createorclear(aamaoutdir, "mylar")
outputfilename = os.path.join(dmylar, "F15-TSF-MYLAR.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
outputfilename = os.path.join(dmylar, "F15-TSF-MYLAR-reflY.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)


layerspresent: {'PATCH-NORM-PLOT', 'MYLAR-CUT', 'TSF-CUT', 'CONSTRUCTION', 'PATCH-CARR-PLOT', 'PATCH-NORM-CUT', 'PATCH-REF', 'PATCH-CARR-CUT', 'HISTORY', 'TSF-PLOT'}
layerspresent: {'PATCH-NORM-PLOT', 'MYLAR-CUT', 'TSF-CUT', 'CONSTRUCTION', 'PATCH-CARR-PLOT', 'PATCH-NORM-CUT', 'PATCH-REF', 'PATCH-CARR-CUT', 'HISTORY', 'TSF-PLOT'}
layerspresent: {'PATCH-NORM-PLOT', 'MYLAR-CUT', 'TSF-CUT', 'CONSTRUCTION', 'PATCH-CARR-PLOT', 'PATCH-NORM-CUT', 'PATCH-REF', 'PATCH-CARR-CUT', 'HISTORY', 'TSF-PLOT'}
dmaxs-tail:  [1.170477633080168e-12, 2.636946534208437e-12, 0.054891012241560164]
dmaxs-tail:  [9.094947017729282e-13, 2.636946534208437e-12, 0.054891012241560164]
dmaxs-tail:  [0.0, 9.094947017729282e-13, 0.054891012241560164]
layerspresent: {'PATCH-NORM-PLOT', 'MYLAR-CUT', 'TSF-CUT', 'CONSTRUCTION', 'PATCH-CARR-PLOT', 'PATCH-NORM-CUT', 'PATCH-REF', 'PATCH-CARR-CUT', 'HISTORY', 'TSF-PLOT'}
dmaxs-tail:  [1.170477633080168e-12, 2.636946534208437e-12, 0.054891012241560164]
dmaxs-tail:  [9.0949470177

In [7]:
#-F15-TSR-TSM-Master-04.dxf. Top surface middle and top surface rear panels. 
#  TSR and TSM cut and plot from either 137cm wide Diax, 144cm wide 170 white or color or 150cm wide ODL06. 
#  Patch cut and patch-plot from 170 color (same as BS).

aamaoutdir = "../dxffiles/aamaout"
dmax = 0.2
b144type = True
 
carrierwingglider = False

inputfilename = "../dxffiles/F15-TSR-TSM-Master-04.dxf"
ddiax = createorclear(aamaoutdir, "diax")



if b144type:
    cutlayers, penlayers = ['TSR-144-CUT'], ['TSR-144-PLOT']
else:
    cutlayers, penlayers = ['TSR-150-CUT'], ['TSR-150-PLOT']
outputfilename = os.path.join(ddiax, "F15-TSR.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
outputfilename = os.path.join(ddiax, "F15-TSR-reflY.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)

if b144type:
    cutlayers, penlayers = ['TSM-144-CUT'], ['TSM-144-PLOT']
else:
    cutlayers, penlayers = ['TSM-150-CUT'], ['TSM-150-PLOT']
outputfilename = os.path.join(ddiax, "F15-TSM.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
outputfilename = os.path.join(ddiax, "F15-TSM-reflY.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)

print("writing to", d170color)
cutlayers, penlayers = ['PATCH-CUT'], ['PATCH-PLOT']
outputfilename = os.path.join(d170color, "F15-PATCH.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
outputfilename = os.path.join(d170color, "F15-PATCH-reflY.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)


layerspresent: {'TSR-144-PLOT', 'PATCH-PLOT', 'TSM-144-CUT', 'TSM-150-CUT', 'TSM-150-PLOT', '0', 'CONSTRUCTION', 'TSR-150-CUT', 'TSR-144-CUT', 'TSM-144-PLOT', 'REFERENCE', 'HISTORY', 'TSR-150-PLOT', 'PATCH-CUT'}
dmaxs-tail:  [1.3191514447337417e-10, 0.0020259419136499047, 0.15062684391013764]
layerspresent: {'TSR-144-PLOT', 'PATCH-PLOT', 'TSM-144-CUT', 'TSM-150-CUT', 'TSM-150-PLOT', '0', 'CONSTRUCTION', 'TSR-150-CUT', 'TSR-144-CUT', 'TSM-144-PLOT', 'REFERENCE', 'HISTORY', 'TSR-150-PLOT', 'PATCH-CUT'}
dmaxs-tail:  [1.3191514447337417e-10, 0.0020259419136499047, 0.15062684391013764]
layerspresent: {'TSR-144-PLOT', 'PATCH-PLOT', 'TSM-144-CUT', 'TSM-150-CUT', 'TSM-150-PLOT', '0', 'CONSTRUCTION', 'TSR-150-CUT', 'TSR-144-CUT', 'TSM-144-PLOT', 'REFERENCE', 'HISTORY', 'TSR-150-PLOT', 'PATCH-CUT'}
layerspresent: {'TSR-144-PLOT', 'PATCH-PLOT', 'TSM-144-CUT', 'TSM-150-CUT', 'TSM-150-PLOT', '0', 'CONSTRUCTION', 'TSR-150-CUT', 'TSR-144-CUT', 'TSM-144-PLOT', 'REFERENCE', 'HISTORY', 'TSR-150-PLOT', '